In [1]:
# import modules needed
import sys
import os
import pandas as pd
import bibtexparser
import re


In [2]:
# set option for visability
pd.set_option('display.max_colwidth', 200)  

In [3]:
# ensure src/ is in the Python path
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(os.path.join(BASE_DIR, "src"))
print(sys.path)

['/opt/homebrew/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python313.zip', '/opt/homebrew/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python3.13', '/opt/homebrew/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/lib-dynload', '', '/Users/celinelangeland/projects/master-thesis/venv/lib/python3.13/site-packages', '/Users/celinelangeland/projects/master-thesis/src']


In [4]:
# import function from file
from retrieval import extract_text_from_pdf

# define path to documents
documents_dir = os.path.join(BASE_DIR, "data", "documents")

# list all PDF files in the folder
pdf_files = [f for f in os.listdir(documents_dir) if f.endswith('.pdf')]
pdf_files

['doc5.pdf',
 'doc4.pdf',
 'doc6.pdf',
 'doc7.pdf',
 'doc3.pdf',
 'doc2.pdf',
 'doc1.pdf']

In [5]:
# list to store the text for each document
texts = []

# loop through all PDF files and extract text
for pdf_file in pdf_files:
    pdf_path = os.path.join(documents_dir, pdf_file)
    df_text = extract_text_from_pdf(pdf_path)
    
    # combine the text content from all pages 
    combined_content = df_text['content'].str.cat(sep=' ')
    
    # extract only doc name
    doc_name = os.path.splitext(pdf_file)[0]

    # create df for document
    df_text = pd.DataFrame([[combined_content, doc_name]], columns=["content", "file"])
    
    # append df to list
    texts.append(df_text)

# concatenate all individual dfs into one combined df
df_text = pd.concat(texts, ignore_index=True)
df_text.head(8)

,content,file
0,Journalism Studies\nISSN: (Print) (Online) Journal homepage: www.tandfonline.com/journals/rjos20\nIntroduction: Underﬆanding Roots and Betweenness\nDeﬁning Safety of Journaliﬆs as a Sub-ﬁeld of Re...,doc5
1,Protocol\nOptimized protocol for conditioned place\navoidance learning in juvenile zebraﬁsh\nConditioned place avoidance assays are broadly used in mammals to study different cognitive\naspects of...,doc4
2,Journal of Energy Storage 72 (2023) 108202\nAvailable online 20 July 2023\n2352-152X/© 2023 The Authors. Published by Elsevier Ltd. This is an open access article under the CC BY-NC-ND license (ht...,doc6
3,"Genetic Algorithms For Tightening Security\n1stFabrizio Palumbo\nArtiﬁcial Intelligence Lab (AI Lab),\nInstitutt for informasjonsteknologi\nOslo Metropolitan University, Oslo, Norway\nfabrizio@osl...",doc7
4,"Journalism Practice\nISSN: (Print) (Online) Journal homepage: www.tandfonline.com/journals/rjop20\nHow (not to) Run an AI Project in Inveﬆigative\nJournalism\nM. Fridman, R. Krøvel & F. Palumbo\nT...",doc3
5,Seminars\n in\n Cell\n &\n Developmental\n Biology\n 78\n (2018)\n 130–139\nContents\n lists\n available\n at\n ScienceDirect\nSeminars\n in\n Cell\n &\n Developmental\n Biology\nj\n ourna\n l\n h...,doc2
6,"Fore et al., Sci. Adv. 2020; 6 : eaaz3173 4 September 2020\nSCIE N C E A D V A NCES | RESEA R CH A RT ICL E\n1 of 17\nD E V E LOPM ENTAL NE U ROS C IE NC E\nFunctional properties of habenular...",doc1


In [6]:
# define the path to metadata
metadata_dir = os.path.join(BASE_DIR, "data", "metadata")

# list all BibTeX files in the
bib_files = [f for f in os.listdir(metadata_dir) if f.endswith('.bib')]
bib_files

['doc4.bib',
 'doc5.bib',
 'doc7.bib',
 'doc6.bib',
 'doc2.bib',
 'doc3.bib',
 'doc1.bib']

In [7]:
def load_bibtex(bibtex_file_path):
    ''' 
    Parses a single BibTeX file and returns a df
    '''
    with open(bibtex_file_path, 'r') as bibtex_file:
        
        # parse the BibTeX file
        bib_database = bibtexparser.load(bibtex_file)
    
    # convert the BibTeX entries into a list of dictionaries
    bib_entries = bib_database.entries
    
    # convert list of dictionaries into a df
    df_bibtex = pd.DataFrame(bib_entries)
    
    return df_bibtex

In [8]:
# list to store dfs
bib_dataframes = []

# loop through all BibTeX files and load them into dfs
for bib_file in bib_files:
    bib_file_path = os.path.join(metadata_dir, bib_file)
    df_bibtex = load_bibtex(bib_file_path)
    
    # extract only doc name
    doc_name = os.path.splitext(bib_file)[0]
    
    # add column to identify the source BibTeX file
    df_bibtex['file'] = doc_name
    
    # append the df to the list
    bib_dataframes.append(df_bibtex)

# concatenate all the dfs into one combined df
df_bibtex = pd.concat(bib_dataframes, ignore_index=True)
df_bibtex.head(8)


,abstract,keywords,author,url,doi,issn,year,pages,number,volume,journal,title,ENTRYTYPE,ID,file,eprint,publisher,booktitle,note
0,"Summary\nConditioned place avoidance assays are broadly used in mammals to study different cognitive aspects of operant learning. Here, we introduce a series of experimental designs for training j...","Model Organisms, Neuroscience, Behavior",Fabrizio Palumbo and Bram Serneels and Emre Yaksi,https://www.sciencedirect.com/science/article/pii/S2666166721001726,https://doi.org/10.1016/j.xpro.2021.100465,2666-1667,2021,100465,2,2,STAR Protocols,Optimized protocol for conditioned place avoidance learning in juvenile zebrafish,article,PALUMBO2021100465,doc4,NaN,NaN,NaN,NaN
1,NaN,NaN,"Roy Krøvel, Fabrizio Palumbo and Kristin Skare Orgeret",\n\nhttps://doi.org/10.1080/1461670X.2023.2206494\n\n\n,10.1080/1461670X.2023.2206494,NaN,2023,825--837,7,24,Journalism Studies,Introduction: Understanding Roots and Betweenness Defining Safety of Journalists as a Sub-field of Research. Reading between the Lines,article,Krøvel19052023,doc5,\n\nhttps://doi.org/10.1080/1461670X.2023.2206494\n\n\n,Routledge,NaN,NaN
2,NaN,Wireless communication;Databases;Operating systems;Delay effects;Quality of service;Reconnaissance;Web servers,"Palumbo, Fabrizio and Buji, Adam and Yazidi, Anis and Haugerud, Hårek",NaN,10.23919/WMNC56391.2022.9954297,NaN,2022,62-69,,,NaN,Genetic Algorithms For Tightening Security,inproceedings,9954297,doc7,NaN,NaN,2022 14th IFIP Wireless and Mobile Networking Conference (WMNC),NaN
3,"The combination of local renewable energy production, dynamic loads, and multiple energy storage systems with different dynamics requires sophisticated control systems to maximize the energy cost ...","Reinforcement learning, COST-WINNERS, Thermal energy storage, Multiple ESS control, Smart warehouse",Sven Myrdahl Opalic and Fabrizio Palumbo and Morten Goodwin and Lei Jiao and Henrik Kofoed Nielsen and Mohan Lal Kolhe,https://www.sciencedirect.com/science/article/pii/S2352152X23015992,https://doi.org/10.1016/j.est.2023.108202,2352-152X,2023,108202,NaN,72,Journal of Energy Storage,COST-WINNERS: COST reduction WIth Neural NEtworks-based augmented Random Search for simultaneous thermal and electrical energy storage control,article,OPALIC2023108202,doc6,NaN,NaN,NaN,NaN
4,The habenula is a brain region that has gained increasing popularity over the recent years due to its role in processing value-related and experience-dependent information with a strong link to de...,"Habenula, Prediction error, Mood disorders, Social behavior, Addiction, Spontaneous activity, Ongoing activity, Default mode network, Sensory activity, Sensory processing, Asymmetry, Learning, Neu...",Stephanie Fore and Fabrizio Palumbo and Robbrecht Pelgrims and Emre Yaksi,https://www.sciencedirect.com/science/article/pii/S1084952116304372,https://doi.org/10.1016/j.semcdb.2017.08.019,1084-9521,2018,130-139,NaN,78,Seminars in Cell & Developmental Biology,Information processing in the vertebrate habenula,article,FORE2018130,doc2,NaN,NaN,NaN,The cancer secretome and secreted biomarkers The vertebrate habenula
5,NaN,NaN,"M. Fridman, R. Krøvel and F. Palumbo",\n\nhttps://doi.org/10.1080/17512786.2023.2253797\n\n\n,10.1080/17512786.2023.2253797,NaN,2023,1--18,0,0,Journalism Practice,How (not to) Run an AI Project in Investigative Journalism,article,Fridman04092023,doc3,\n\nhttps://doi.org/10.1080/17512786.2023.2253797\n\n\n,Routledge,NaN,NaN
6,"Sequential neurogenesis determines the functional properties of habenular neurons. The developing brain undergoes drastic alterations. Here, we investigated developmental changes in the habenula, ...",NaN,Stephanie Fore and Francisca Acuña-Hinrichsen and Kadir Aytac Mutlu and Ewelina Magdalena Bartoszek and Bram Serneels and Nicholas Guy Faturos and Khac Thanh Phong Chau and Mehmet Ilyas Cos...,https://www.science.org/doi/abs/10.1126/sciadv.aaz3173,10.1126/sciadv.aaz3173,NaN,2020,eaaz3173,36,6,Science Advances,Functional properti

In [9]:
# select only relevant columns
df_bibtex = df_bibtex[['file', 'title', 'author', 'year', 'number', 'volume', 'journal', 'ENTRYTYPE', 'doi']]
df_bibtex.head(8)

,file,title,author,year,number,volume,journal,ENTRYTYPE,doi
0,doc4,Optimized protocol for conditioned place avoidance learning in juvenile zebrafish,Fabrizio Palumbo and Bram Serneels and Emre Yaksi,2021,2,2,STAR Protocols,article,https://doi.org/10.1016/j.xpro.2021.100465
1,doc5,Introduction: Understanding Roots and Betweenness Defining Safety of Journalists as a Sub-field of Research. Reading between the Lines,"Roy Krøvel, Fabrizio Palumbo and Kristin Skare Orgeret",2023,7,24,Journalism Studies,article,10.1080/1461670X.2023.2206494
2,doc7,Genetic Algorithms For Tightening Security,"Palumbo, Fabrizio and Buji, Adam and Yazidi, Anis and Haugerud, Hårek",2022,,,NaN,inproceedings,10.23919/WMNC56391.2022.9954297
3,doc6,COST-WINNERS: COST reduction WIth Neural NEtworks-based augmented Random Search for simultaneous thermal and electrical energy storage control,Sven Myrdahl Opalic and Fabrizio Palumbo and Morten Goodwin and Lei Jiao and Henrik Kofoed Nielsen and Mohan Lal Kolhe,2023,NaN,72,Journal of Energy Storage,article,https://doi.org/10.1016/j.est.2023.108202
4,doc2,Information processing in the vertebrate habenula,Stephanie Fore and Fabrizio Palumbo and Robbrecht Pelgrims and Emre Yaksi,2018,NaN,78,Seminars in Cell & Developmental Biology,article,https://doi.org/10.1016/j.semcdb.2017.08.019
5,doc3,How (not to) Run an AI Project in Investigative Journalism,"M. Fridman, R. Krøvel and F. Palumbo",2023,0,0,Journalism Practice,article,10.1080/17512786.2023.2253797
6,doc1,Functional properties of habenular neurons are determined by developmental stage and sequential neurogenesis,Stephanie Fore and Francisca Acuña-Hinrichsen and Kadir Aytac Mutlu and Ewelina Magdalena Bartoszek and Bram Serneels and Nicholas Guy Faturos and Khac Thanh Phong Chau and Mehmet Ilyas Cos...,2020,36,6,Science Advances,article,10.1126/sciadv.aaz3173


In [ ]:
# merge the df_text and df_bibtex on the index columns
df_combined = pd.merge(df_text, df_bibtex, on='file', how='left')
df_combined.head()

,content,file,title,author,year,number,volume,journal,ENTRYTYPE,doi
0,Journalism Studies\nISSN: (Print) (Online) Journal homepage: www.tandfonline.com/journals/rjos20\nIntroduction: Underﬆanding Roots and Betweenness\nDeﬁning Safety of Journaliﬆs as a Sub-ﬁeld of Re...,doc5,Introduction: Understanding Roots and Betweenness Defining Safety of Journalists as a Sub-field of Research. Reading between the Lines,"Roy Krøvel, Fabrizio Palumbo and Kristin Skare Orgeret",2023,7,24,Journalism Studies,article,10.1080/1461670X.2023.2206494
1,Protocol\nOptimized protocol for conditioned place\navoidance learning in juvenile zebraﬁsh\nConditioned place avoidance assays are broadly used in mammals to study different cognitive\naspects of...,doc4,Optimized protocol for conditioned place avoidance learning in juvenile zebrafish,Fabrizio Palumbo and Bram Serneels and Emre Yaksi,2021,2,2,STAR Protocols,article,https://doi.org/10.1016/j.xpro.2021.100465
2,Journal of Energy Storage 72 (2023) 108202\nAvailable online 20 July 2023\n2352-152X/© 2023 The Authors. Published by Elsevier Ltd. This is an open access article under the CC BY-NC-ND license (ht...,doc6,COST-WINNERS: COST reduction WIth Neural NEtworks-based augmented Random Search for simultaneous thermal and electrical energy storage control,Sven Myrdahl Opalic and Fabrizio Palumbo and Morten Goodwin and Lei Jiao and Henrik Kofoed Nielsen and Mohan Lal Kolhe,2023,NaN,72,Journal of Energy Storage,article,https://doi.org/10.1016/j.est.2023.108202
3,"Genetic Algorithms For Tightening Security\n1stFabrizio Palumbo\nArtiﬁcial Intelligence Lab (AI Lab),\nInstitutt for informasjonsteknologi\nOslo Metropolitan University, Oslo, Norway\nfabrizio@osl...",doc7,Genetic Algorithms For Tightening Security,"Palumbo, Fabrizio and Buji, Adam and Yazidi, Anis and Haugerud, Hårek",2022,,,NaN,inproceedings,10.23919/WMNC56391.2022.9954297
4,"Journalism Practice\nISSN: (Print) (Online) Journal homepage: www.tandfonline.com/journals/rjop20\nHow (not to) Run an AI Project in Inveﬆigative\nJournalism\nM. Fridman, R. Krøvel & F. Palumbo\nT...",doc3,How (not to) Run an AI Project in Investigative Journalism,"M. Fridman, R. Krøvel and F. Palumbo",2023,0,0,Journalism Practice,article,10.1080/17512786.2023.2253797


In [11]:
df_combined = df_combined[['title', 'author', 'year', 'number', 'volume', 'journal', 'ENTRYTYPE', 'content', 'doi']]
df_combined

,title,author,year,number,volume,journal,ENTRYTYPE,content,doi
0,Introduction: Understanding Roots and Betweenness Defining Safety of Journalists as a Sub-field of Research. Reading between the Lines,"Roy Krøvel, Fabrizio Palumbo and Kristin Skare Orgeret",2023,7,24,Journalism Studies,article,Journalism Studies\nISSN: (Print) (Online) Journal homepage: www.tandfonline.com/journals/rjos20\nIntroduction: Underﬆanding Roots and Betweenness\nDeﬁning Safety of Journaliﬆs as a Sub-ﬁeld of Re...,10.1080/1461670X.2023.2206494
1,Optimized protocol for conditioned place avoidance learning in juvenile zebrafish,Fabrizio Palumbo and Bram Serneels and Emre Yaksi,2021,2,2,STAR Protocols,article,Protocol\nOptimized protocol for conditioned place\navoidance learning in juvenile zebraﬁsh\nConditioned place avoidance assays are broadly used in mammals to study different cognitive\naspects of...,https://doi.org/10.1016/j.xpro.2021.100465
2,COST-WINNERS: COST reduction WIth Neural NEtworks-based augmented Random Search for simultaneous thermal and electrical energy storage control,Sven Myrdahl Opalic and Fabrizio Palumbo and Morten Goodwin and Lei Jiao and Henrik Kofoed Nielsen and Mohan Lal Kolhe,2023,NaN,72,Journal of Energy Storage,article,Journal of Energy Storage 72 (2023) 108202\nAvailable online 20 July 2023\n2352-152X/© 2023 The Authors. Published by Elsevier Ltd. This is an open access article under the CC BY-NC-ND license (ht...,https://doi.org/10.1016/j.est.2023.108202
3,Genetic Algorithms For Tightening Security,"Palumbo, Fabrizio and Buji, Adam and Yazidi, Anis and Haugerud, Hårek",2022,,,NaN,inproceedings,"Genetic Algorithms For Tightening Security\n1stFabrizio Palumbo\nArtiﬁcial Intelligence Lab (AI Lab),\nInstitutt for informasjonsteknologi\nOslo Metropolitan University, Oslo, Norway\nfabrizio@osl...",10.23919/WMNC56391.2022.9954297
4,How (not to) Run an AI Project in Investigative Journalism,"M. Fridman, R. Krøvel and F. Palumbo",2023,0,0,Journalism Practice,article,"Journalism Practice\nISSN: (Print) (Online) Journal homepage: www.tandfonline.com/journals/rjop20\nHow (not to) Run an AI Project in Inveﬆigative\nJournalism\nM. Fridman, R. Krøvel & F. Palumbo\nT...",10.1080/17512786.2023.2253797
5,Information processing in the vertebrate habenula,Stephanie Fore and Fabrizio Palumbo and Robbrecht Pelgrims and Emre Yaksi,2018,NaN,78,Seminars in Cell & Developmental Biology,article,Seminars\n in\n Cell\n &\n Developmental\n Biology\n 78\n (2018)\n 130–139\nContents\n lists\n available\n at\n ScienceDirect\nSeminars\n in\n Cell\n &\n Developmental\n Biology\nj\n ourna\n l\n h...,https://doi.org/10.1016/j.semcdb.2017.08.019
6,Functional properties of habenular neurons are determined by developmental stage and sequential neurogenesis,Stephanie Fore and Francisca Acuña-Hinrichsen and Kadir Aytac Mutlu and Ewelina Magdalena Bartoszek and Bram Serneels and Nicholas Guy Faturos and Khac Thanh Phong Chau and Mehmet Ilyas Cos...,2020,36,6,Science Advances,article,"Fore et al., Sci. Adv. 2020; 6 : eaaz3173 4 September 2020\nSCIE N C E A D V A NCES | RESEA R CH A RT ICL E\n1 of 17\nD E V E LOPM ENTAL NE U ROS C IE NC E\nFunctional properties of habenular...",10.1126/sciadv.aaz3173


In [12]:
# rename columns
df_combined = df_combined.rename(columns={'ENTRYTYPE':'type', 'author':'authors', 'year':'year_published'})
df_combined

,title,authors,year_published,number,volume,journal,type,content,doi
0,Introduction: Understanding Roots and Betweenness Defining Safety of Journalists as a Sub-field of Research. Reading between the Lines,"Roy Krøvel, Fabrizio Palumbo and Kristin Skare Orgeret",2023,7,24,Journalism Studies,article,Journalism Studies\nISSN: (Print) (Online) Journal homepage: www.tandfonline.com/journals/rjos20\nIntroduction: Underﬆanding Roots and Betweenness\nDeﬁning Safety of Journaliﬆs as a Sub-ﬁeld of Re...,10.1080/1461670X.2023.2206494
1,Optimized protocol for conditioned place avoidance learning in juvenile zebrafish,Fabrizio Palumbo and Bram Serneels and Emre Yaksi,2021,2,2,STAR Protocols,article,Protocol\nOptimized protocol for conditioned place\navoidance learning in juvenile zebraﬁsh\nConditioned place avoidance assays are broadly used in mammals to study different cognitive\naspects of...,https://doi.org/10.1016/j.xpro.2021.100465
2,COST-WINNERS: COST reduction WIth Neural NEtworks-based augmented Random Search for simultaneous thermal and electrical energy storage control,Sven Myrdahl Opalic and Fabrizio Palumbo and Morten Goodwin and Lei Jiao and Henrik Kofoed Nielsen and Mohan Lal Kolhe,2023,NaN,72,Journal of Energy Storage,article,Journal of Energy Storage 72 (2023) 108202\nAvailable online 20 July 2023\n2352-152X/© 2023 The Authors. Published by Elsevier Ltd. This is an open access article under the CC BY-NC-ND license (ht...,https://doi.org/10.1016/j.est.2023.108202
3,Genetic Algorithms For Tightening Security,"Palumbo, Fabrizio and Buji, Adam and Yazidi, Anis and Haugerud, Hårek",2022,,,NaN,inproceedings,"Genetic Algorithms For Tightening Security\n1stFabrizio Palumbo\nArtiﬁcial Intelligence Lab (AI Lab),\nInstitutt for informasjonsteknologi\nOslo Metropolitan University, Oslo, Norway\nfabrizio@osl...",10.23919/WMNC56391.2022.9954297
4,How (not to) Run an AI Project in Investigative Journalism,"M. Fridman, R. Krøvel and F. Palumbo",2023,0,0,Journalism Practice,article,"Journalism Practice\nISSN: (Print) (Online) Journal homepage: www.tandfonline.com/journals/rjop20\nHow (not to) Run an AI Project in Inveﬆigative\nJournalism\nM. Fridman, R. Krøvel & F. Palumbo\nT...",10.1080/17512786.2023.2253797
5,Information processing in the vertebrate habenula,Stephanie Fore and Fabrizio Palumbo and Robbrecht Pelgrims and Emre Yaksi,2018,NaN,78,Seminars in Cell & Developmental Biology,article,Seminars\n in\n Cell\n &\n Developmental\n Biology\n 78\n (2018)\n 130–139\nContents\n lists\n available\n at\n ScienceDirect\nSeminars\n in\n Cell\n &\n Developmental\n Biology\nj\n ourna\n l\n h...,https://doi.org/10.1016/j.semcdb.2017.08.019
6,Functional properties of habenular neurons are determined by developmental stage and sequential neurogenesis,Stephanie Fore and Francisca Acuña-Hinrichsen and Kadir Aytac Mutlu and Ewelina Magdalena Bartoszek and Bram Serneels and Nicholas Guy Faturos and Khac Thanh Phong Chau and Mehmet Ilyas Cos...,2020,36,6,Science Advances,article,"Fore et al., Sci. Adv. 2020; 6 : eaaz3173 4 September 2020\nSCIE N C E A D V A NCES | RESEA R CH A RT ICL E\n1 of 17\nD E V E LOPM ENTAL NE U ROS C IE NC E\nFunctional properties of habenular...",10.1126/sciadv.aaz3173


In [ ]:
#df_combined.to_csv('df_combined.csv')

,title,authors,year_published,number,volume,journal,type,content,doi
0,Introduction: Understanding Roots and Betweenness Defining Safety of Journalists as a Sub-field of Research. Reading between the Lines,"Roy Krøvel, Fabrizio Palumbo and Kristin Skare Orgeret",2023,7,24,Journalism Studies,article,Journalism Studies\nISSN: (Print) (Online) Journal homepage: www.tandfonline.com/journals/rjos20\nIntroduction: Underﬆanding Roots and Betweenness\nDeﬁning Safety of Journaliﬆs as a Sub-ﬁeld of Re...,10.1080/1461670X.2023.2206494
1,Optimized protocol for conditioned place avoidance learning in juvenile zebrafish,Fabrizio Palumbo and Bram Serneels and Emre Yaksi,2021,2,2,STAR Protocols,article,Protocol\nOptimized protocol for conditioned place\navoidance learning in juvenile zebraﬁsh\nConditioned place avoidance assays are broadly used in mammals to study different cognitive\naspects of...,https://doi.org/10.1016/j.xpro.2021.100465
2,COST-WINNERS: COST reduction WIth Neural NEtworks-based augmented Random Search for simultaneous thermal and electrical energy storage control,Sven Myrdahl Opalic and Fabrizio Palumbo and Morten Goodwin and Lei Jiao and Henrik Kofoed Nielsen and Mohan Lal Kolhe,2023,missing,72,Journal of Energy Storage,article,Journal of Energy Storage 72 (2023) 108202\nAvailable online 20 July 2023\n2352-152X/© 2023 The Authors. Published by Elsevier Ltd. This is an open access article under the CC BY-NC-ND license (ht...,https://doi.org/10.1016/j.est.2023.108202
3,Genetic Algorithms For Tightening Security,"Palumbo, Fabrizio and Buji, Adam and Yazidi, Anis and Haugerud, Hårek",2022,,,missing,inproceedings,"Genetic Algorithms For Tightening Security\n1stFabrizio Palumbo\nArtiﬁcial Intelligence Lab (AI Lab),\nInstitutt for informasjonsteknologi\nOslo Metropolitan University, Oslo, Norway\nfabrizio@osl...",10.23919/WMNC56391.2022.9954297
4,How (not to) Run an AI Project in Investigative Journalism,"M. Fridman, R. Krøvel and F. Palumbo",2023,0,0,Journalism Practice,article,"Journalism Practice\nISSN: (Print) (Online) Journal homepage: www.tandfonline.com/journals/rjop20\nHow (not to) Run an AI Project in Inveﬆigative\nJournalism\nM. Fridman, R. Krøvel & F. Palumbo\nT...",10.1080/17512786.2023.2253797
5,Information processing in the vertebrate habenula,Stephanie Fore and Fabrizio Palumbo and Robbrecht Pelgrims and Emre Yaksi,2018,missing,78,Seminars in Cell & Developmental Biology,article,Seminars\n in\n Cell\n &\n Developmental\n Biology\n 78\n (2018)\n 130–139\nContents\n lists\n available\n at\n ScienceDirect\nSeminars\n in\n Cell\n &\n Developmental\n Biology\nj\n ourna\n l\n h...,https://doi.org/10.1016/j.semcdb.2017.08.019
6,Functional properties of habenular neurons are determined by developmental stage and sequential neurogenesis,Stephanie Fore and Francisca Acuña-Hinrichsen and Kadir Aytac Mutlu and Ewelina Magdalena Bartoszek and Bram Serneels and Nicholas Guy Faturos and Khac Thanh Phong Chau and Mehmet Ilyas Cos...,2020,36,6,Science Advances,article,"Fore et al., Sci. Adv. 2020; 6 : eaaz3173 4 September 2020\nSCIE N C E A D V A NCES | RESEA R CH A RT ICL E\n1 of 17\nD E V E LOPM ENTAL NE U ROS C IE NC E\nFunctional properties of habenular...",10.1126/sciadv.aaz3173
